<a href="https://colab.research.google.com/github/wangbxj1234/offset_pt_try/blob/main/torch_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset


def load_data(partition):
    all_data = []
    all_label = []
    for h5_name in glob.glob('/content/drive/MyDrive/PAConv-main/modelnet40_ply_hdf5_2048/ply_data_%s*.h5' % partition):
        f = h5py.File(h5_name, mode='r')
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label


def pc_normalize(pc):###用了
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    pc = pc / m
    return pc


def translate_pointcloud(pointcloud):####用了
    xyz1 = np.random.uniform(low=2. / 3., high=3. / 2., size=[3])####从一个均匀分布[low,high)中随机采样， size: 输出样本数目为3
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])###########从一个均匀分布[low,high)中随机采样， size: 输出样本数目为3

    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')#######数据增强
    return translated_pointcloud


def jitter_pointcloud(pointcloud, sigma=0.01, clip=0.02):####没用
    N, C = pointcloud.shape
    pointcloud += np.clip(sigma * np.random.randn(N, C), -1 * clip, clip)
    return pointcloud


class ModelNet40(Dataset):
    def __init__(self, num_points, partition='train', pt_norm=False):
        self.data, self.label = load_data(partition)
        self.num_points = num_points
        self.partition = partition
        self.pt_norm = pt_norm

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]            #####就取了前1024个点，不出问题才怪
        label = self.label[item]
        if self.partition == 'train':
            if self.pt_norm:
                pointcloud = pc_normalize(pointcloud)
            pointcloud = translate_pointcloud(pointcloud)
            np.random.shuffle(pointcloud)  # shuffle the order of pts
        return pointcloud, label

    def __len__(self):
        return self.data.shape[0]#########原始data是：9840，2048，3， 从2048中取了1024个点
    

In [24]:
set=ModelNet40(partition='train', num_points=1024, pt_norm=True)

In [38]:
set[9839][0].shape###########9840,1024,3(point cloud)

(1024, 3)

In [42]:
set[9839][1].shape############9840,1 (label)

(1,)

In [21]:
len(set) 

9840